In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torch_geometric.loader import DataLoader
from src.training.train import train
from src.model.gae import GAE
from src.data.loader import GraphDataset
from src.utils import commons

device = commons.get_device()
# device = 'cuda'
torch.cuda.empty_cache()
dataset = GraphDataset()
config = commons.get_config('configs/reduced.yaml')
model = GAE(config=config['model'], num_nodes = dataset.num_nodes).to(device)

# optimizer
if config['training']['optimizer']['type'] == 'Adam':
    from torch.optim.adam import Adam
    optimizer = Adam(model.parameters(), lr=config['training']['optimizer']['learning_rate'])
elif config['training']['optimizer']['type'] == 'AdamW':
    from torch.optim.adamw import AdamW
    optimizer = AdamW(model.parameters(), lr=config['training']['optimizer']['learning_rate'])
else:
    raise ValueError(f"Invalid optimizer: {config['training']['optimizer']['type']}")

# scheduler 
if config['training']['scheduler']['type'] == 'StepLR':
    from torch.optim.lr_scheduler import StepLR
    scheduler = StepLR(optimizer, step_size=config['training']['scheduler']['step_size'], gamma=config['training']['scheduler']['gamma'])
elif config['training']['scheduler']['type'] == 'CosineAnnealingLR':
    from torch.optim.lr_scheduler import CosineAnnealingLR
    scheduler = CosineAnnealingLR(optimizer, T_max=config['training']['epochs'])
else:
    raise ValueError(f"Invalid scheduler: {config['training']['scheduler']['type']}")

train_loader = DataLoader(dataset=dataset, 
                          batch_size=config['training']['batch_size'], 
                          shuffle=False,
                          num_workers=config['config']['num_workers'])

train_history = train(model=model, 
                      optimizer=optimizer,
                      device=device,
                      scheduler=scheduler, 
                      train_loader=train_loader, 
                      config=config['training'])

Sequential(
  (encoder_layer_0): Linear(in_features=493536, out_features=2048, bias=True)
  (encoder_layer_0_relu): ReLU()
  (encoder_layer_1): Linear(in_features=2048, out_features=1024, bias=True)
  (encoder_layer_1_relu): ReLU()
  (encoder_layer_2): Linear(in_features=1024, out_features=256, bias=True)
  (encoder_layer_2_relu): ReLU()
  (encoder_layer_3): Linear(in_features=256, out_features=64, bias=True)
  (encoder_layer_3_relu): ReLU()
  (encoder_layer_4): Linear(in_features=64, out_features=16, bias=True)
  (encoder_layer_4_relu): ReLU()
  (encoder_layer_latent_dim): Linear(in_features=16, out_features=2, bias=True)
  (encoder_layer_latent_dim_relu): ReLU()
)
Sequential(
  (decoder_layer_latent_dim): Linear(in_features=2, out_features=16, bias=True)
  (decoder_layer_latent_dim_relu): ReLU()
  (decoder_layer_0): Linear(in_features=16, out_features=64, bias=True)
  (decoder_layer_0_relu): ReLU()
  (decoder_layer_1): Linear(in_features=64, out_features=256, bias=True)
  (decoder_la

AssertionError: Torch not compiled with CUDA enabled

In [5]:
list = [1,2,3,4,5]
list[1:]

[2, 3, 4, 5]

In [1]:

from torch_geometric.nn import GCNConv, ChebConv, GATConv, GMMConv
from src.data.loader import GraphDataset
import torch.nn as nn
# Create dataset and get a batch
dataset = GraphDataset()
batch = dataset[0]

# Try GCN with edge weights
# test_model = GCNConv(2, 32)
# batch.x = test_model(batch.x, batch.edge_index, batch.edge_weight)

encoder = nn.ModuleList()
decoder = nn.ModuleList()

encoder.append(GCNConv(2, 32))
decoder.append(GCNConv(32, 2))

for i in range(3):
    encoder.append(GCNConv(32 - 2*i, 32 - 2 * (i+1)))
    decoder.append(GCNConv(32 - 2 * (i+1), 32 - 2*i))

decoder = decoder[-1::-1]

d:\TA\TA2\GAE-ROM\src\data\loader.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(self.edge_list, dtype=torch.long),
d:\TA\TA2\GAE-ROM\src\data\loader.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(self.edge_features, dtype=torch.float32),
d:\TA\TA2\GAE-ROM\src\data\loader.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weight = torch.tensor(self.edge_weights, dtype=torch.float32))


In [21]:
for model in encoder:
    batch.x = model(batch.x, batch.edge_index, batch.edge_weight)

for model in decoder:
    batch.x = model(batch.x, batch.edge_index, batch.edge_weight)
# batch.x = test_module(batch.x, batch.edge_index, batch.edge_weight)
batch.x

tensor([[-3.3373, -4.2184],
        [-2.5727, -3.5770],
        [-2.7493, -3.7444],
        ...,
        [-3.0141, -5.2052],
        [-4.0269, -5.0360],
        [-4.8975, -5.9311]], grad_fn=<AddBackward0>)

In [2]:
import torch_geometric.nn.pool as pooling

topk = pooling.TopKPooling(in_channels=2, ratio=0.5, nonlinearity='relu')

TopKPooling

In [ ]:
print(batch.x.shape)
print(batch.edge_index.shape)
print(batch.edge_attr.shape)
pooled_x, pooled_edge_index, pooled_edge_attr, batch_index, perm, score = topk(x = batch.x, edge_index = batch.edge_index, edge_attr = batch.edge_attr)

print(pooled_x.shape)
print(pooled_edge_index.shape)
print(pooled_edge_attr.shape)
print(batch_index.shape)
print(perm.shape)
print(score.shape)

torch.Size([30846, 2])
torch.Size([2, 46015])
torch.Size([46015])
torch.Size([3085, 2])
torch.Size([2, 4424])
torch.Size([4424])
torch.Size([3085])
torch.Size([3085])
torch.Size([3085])


In [1]:
pooled_x.shape

NameError: name 'pooled_x' is not defined

EdgePooling

In [1]:
from torch_geometric.nn.pool import EdgePooling
from torch_geometric.nn import GCNConv, ChebConv, GATConv, GMMConv
from src.data.loader import GraphDataset
import torch.nn as nn
# Create dataset and get a batch
dataset = GraphDataset()
batch = dataset[0]
edgepool = EdgePooling(in_channels=2, dropout=0)

d:\TA\TA2\GAE-ROM\src\data\loader.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(self.edge_list, dtype=torch.long),
d:\TA\TA2\GAE-ROM\src\data\loader.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(self.edge_features, dtype=torch.float32),
d:\TA\TA2\GAE-ROM\src\data\loader.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weight = torch.tensor(self.edge_weights, dtype=torch.float32))


In [9]:
batch.pos

tensor([[2.5045, 2.8414],
        [4.1457, 2.4405],
        [2.9354, 1.7456],
        ...,
        [2.3839, 1.3395],
        [1.0981, 1.1352],
        [1.5320, 1.6905]])

In [12]:
print(type(batch))           # should be <class 'torch_geometric.data.data.Data'>
print(type(batch.batch))     # should be <class 'torch.Tensor'>
print(batch.x.size(0))     # e.g., [num_nodes]


<class 'torch_geometric.data.data.Data'>
<class 'NoneType'>
30846


In [7]:
print(batch.x.shape)
print(batch.edge_index.shape)
print(max(batch.edge_index[1]))
print(batch.edge_attr.shape)
import torch

# If your graph has N nodes, all belong to the same graph, so assign zeros:
batch_vector = torch.zeros(batch.x.size(0), dtype=torch.long)

pooled_x, pooled_edge_index, pooled_batch, unpool_info = edgepool(batch =  batch_vector, x = batch.x, edge_index = batch.edge_index)
# pooled_result = edgepool(batch = batch_vector, x = batch.x, edge_index = batch.edge_index)

print(pooled_x.shape)
print(pooled_edge_index.shape)
print(max(pooled_edge_index[1]))
print(pooled_batch)

torch.Size([30846, 2])
torch.Size([2, 46015])
tensor(30845)
torch.Size([46015])
torch.Size([17306, 2])
torch.Size([2, 46014])
tensor(17305)
tensor([0, 0, 0,  ..., 0, 0, 0])


In [16]:
batch.x

tensor([[46.3758, -0.8608],
        [38.7250, -1.5845],
        [40.6678, -1.4719],
        ...,
        [28.8896, -2.3286],
        [34.1928, -0.3318],
        [34.1816, -0.2373]])

In [15]:
pooled_x

tensor([[ 1.0998e+02,  8.1582e-01],
        [ 1.3889e+02, -2.8769e+00],
        [ 1.1589e+02, -4.7028e+00],
        ...,
        [ 3.5362e+01, -6.6867e-02],
        [ 3.5490e+01,  7.5209e-01],
        [ 3.5759e+01,  1.4351e+00]], grad_fn=<MulBackward0>)

In [ ]:
from src.utils.plot import Plot

plot = Plot(data_dir = "data", save_dir = "output")
plot.plot_tensor(tensor = pooled_x[:,0], data_dim = (1, 2), title = "Pooled X")
